In [13]:
import pandas as pd
from utils import *
from typing import Any

<h1>Cleaning a CSV</h1>

Read in a CSV

In [14]:
# load a dataframe
infile_path = "../data/raw/silver_shelby_county_properties_municipality_breakdown - s12_2021_idb_germantown.csv"
df = pd.read_csv(infile_path)
df

,Owner,Location,PILOT Parcel No,Underlying Parcel Number,Begin Date,End Date,Assmt Taxes,PILOT Contract Amt,PILOT Debt Service Amt,Difference
0,GILL POPLAR GP,6797 HWY 72,IDB0000G000050,G0219000003110,03/17/2018,03/17/2033,"$346,522","$169,314","$44,302","-$132,906"
1,"MID-AMERICA APARTMENTS, LP",6815 POPLAR,IDB0000G00006A,6000050000000D,12/31/2018,12/31/2033,"$16,744",$0,"$4,186","-$12,558"
2,CAMPBELL CLINIC - WR MOB,1400 GERMANTOWN,IDB0000G000070,"G0220000003690, G0220000004560, G0220000004570",12/31/2019,12/30/2027,"$671,422","$123,694","$136,932","-$410,796"
3,CRESTWYN PARTNERS,9280 CRESTWYN HILLS,IDB14000000000,G0243000004890,08/16/2016,08/16/2023,"$119,643","$17,183","$25,615","-$76,845"


In [15]:
# Clean the column headers
df = clean_headers(df, [remove_special_for_words, truncate, snake_case, lower_case])
df.columns

Index(['owner', 'location', 'pilot_parcel_no', 'underlying_parcel_number',
       'begin_date', 'end_date', 'assmt_taxes', 'pilot_contract_amt',
       'pilot_debt_service_amt', 'difference'],
      dtype='object')

In [16]:
# Clean the string columns
string_columns = ["owner", 
                  "location"]
df = clean_columns(df, string_columns, [remove_special_for_words, truncate, lower_case])
df

,owner,location,pilot_parcel_no,underlying_parcel_number,begin_date,end_date,assmt_taxes,pilot_contract_amt,pilot_debt_service_amt,difference
0,gill poplar gp,6797 hwy 72,IDB0000G000050,G0219000003110,03/17/2018,03/17/2033,"$346,522","$169,314","$44,302","-$132,906"
1,mid america apartments lp,6815 poplar,IDB0000G00006A,6000050000000D,12/31/2018,12/31/2033,"$16,744",$0,"$4,186","-$12,558"
2,campbell clinic wr mob,1400 germantown,IDB0000G000070,"G0220000003690, G0220000004560, G0220000004570",12/31/2019,12/30/2027,"$671,422","$123,694","$136,932","-$410,796"
3,crestwyn partners,9280 crestwyn hills,IDB14000000000,G0243000004890,08/16/2016,08/16/2023,"$119,643","$17,183","$25,615","-$76,845"


In [17]:
df.columns

Index(['owner', 'location', 'pilot_parcel_no', 'underlying_parcel_number',
       'begin_date', 'end_date', 'assmt_taxes', 'pilot_contract_amt',
       'pilot_debt_service_amt', 'difference'],
      dtype='object')

In [18]:
# Clean the numeric columns
numeric_columns = ['assmt_taxes', 'pilot_contract_amt',
       'pilot_debt_service_amt', 'difference']
df = clean_columns(df, numeric_columns, [remove_special_for_numbers, convert_only_dash_to_missing])

df

,owner,location,pilot_parcel_no,underlying_parcel_number,begin_date,end_date,assmt_taxes,pilot_contract_amt,pilot_debt_service_amt,difference
0,gill poplar gp,6797 hwy 72,IDB0000G000050,G0219000003110,03/17/2018,03/17/2033,346522,169314,44302,-132906
1,mid america apartments lp,6815 poplar,IDB0000G00006A,6000050000000D,12/31/2018,12/31/2033,16744,0,4186,-12558
2,campbell clinic wr mob,1400 germantown,IDB0000G000070,"G0220000003690, G0220000004560, G0220000004570",12/31/2019,12/30/2027,671422,123694,136932,-410796
3,crestwyn partners,9280 crestwyn hills,IDB14000000000,G0243000004890,08/16/2016,08/16/2023,119643,17183,25615,-76845


In [19]:
# Clean the date columns

# Because this data has an idiosyncratic "end date" value of "End of Loan", 
# we must define a dataset-specific function
def cast_end_date_to_datetime(entry: str | Number):
    try:
        return pd.to_datetime(str(entry))
    except ValueError:
        return entry
df["begin_date"] = pd.to_datetime(df["begin_date"])
df["end_date"] = df["end_date"].apply(lambda x: cast_end_date_to_datetime(x))
df

,owner,location,pilot_parcel_no,underlying_parcel_number,begin_date,end_date,assmt_taxes,pilot_contract_amt,pilot_debt_service_amt,difference
0,gill poplar gp,6797 hwy 72,IDB0000G000050,G0219000003110,2018-03-17,2033-03-17,346522,169314,44302,-132906
1,mid america apartments lp,6815 poplar,IDB0000G00006A,6000050000000D,2018-12-31,2033-12-31,16744,0,4186,-12558
2,campbell clinic wr mob,1400 germantown,IDB0000G000070,"G0220000003690, G0220000004560, G0220000004570",2019-12-31,2027-12-30,671422,123694,136932,-410796
3,crestwyn partners,9280 crestwyn hills,IDB14000000000,G0243000004890,2016-08-16,2023-08-16,119643,17183,25615,-76845


In [20]:
# Clean the list columns

# Pandas always reads in lists as strings, so they will ultimately be converted to a string type, but putting them in lists firsts will allow us to convert them back to lists more quickly the next time we read them in because the list brackets will be included 
# in the output string

list_columns = ["pilot_parcel_no", "underlying_parcel_number"]
for col in list_columns:
    df[col] = df[col].apply(lambda x: str(x).split(","))
df

,owner,location,pilot_parcel_no,underlying_parcel_number,begin_date,end_date,assmt_taxes,pilot_contract_amt,pilot_debt_service_amt,difference
0,gill poplar gp,6797 hwy 72,[IDB0000G000050],[G0219000003110],2018-03-17,2033-03-17,346522,169314,44302,-132906
1,mid america apartments lp,6815 poplar,[IDB0000G00006A],[6000050000000D],2018-12-31,2033-12-31,16744,0,4186,-12558
2,campbell clinic wr mob,1400 germantown,[IDB0000G000070],"[G0220000003690, G0220000004560, G0220000004...",2019-12-31,2027-12-30,671422,123694,136932,-410796
3,crestwyn partners,9280 crestwyn hills,[IDB14000000000],[G0243000004890],2016-08-16,2023-08-16,119643,17183,25615,-76845


In [21]:
# Create a unique index column based on the values in the identifying column
# Note: Ids are based on a deterministic hash, which means they are 
# reproducible even if the function is run by a different user at a different time,
# as long as the string in the identifying column is the same. 

identifying_columns = ["owner"]
index_name = "owner_id"
df = set_unique_index(df=df, columns_to_hash = identifying_columns, index_length_limit=12, index_name=index_name)
df

,owner,location,pilot_parcel_no,underlying_parcel_number,begin_date,end_date,assmt_taxes,pilot_contract_amt,pilot_debt_service_amt,difference
owner_id,,,,,,,,,,
fa0c449b658b,gill poplar gp,6797 hwy 72,[IDB0000G000050],[G0219000003110],2018-03-17,2033-03-17,346522,169314,44302,-132906
aeaf02f07dc9,mid america apartments lp,6815 poplar,[IDB0000G00006A],[6000050000000D],2018-12-31,2033-12-31,16744,0,4186,-12558
3f73c512e692,campbell clinic wr mob,1400 germantown,[IDB0000G000070],"[G0220000003690, G0220000004560, G0220000004...",2019-12-31,2027-12-30,671422,123694,136932,-410796
c90d7ab8918a,crestwyn partners,9280 crestwyn hills,[IDB14000000000],[G0243000004890],2016-08-16,2023-08-16,119643,17183,25615,-76845


In [22]:
# Check the data types of each variable
df.dtypes

owner                               object
location                            object
pilot_parcel_no                     object
underlying_parcel_number            object
begin_date                  datetime64[ns]
end_date                    datetime64[ns]
assmt_taxes                         object
pilot_contract_amt                  object
pilot_debt_service_amt              object
difference                          object
dtype: object

In [23]:
# Fix the data type of each variable

names_to_types = {'owner': 'string',
                  'location': 'string',
                  'pilot_parcel_no': 'string', 
                  'underlying_parcel_number': 'string',
                  'begin_date': 'datetime64[ns]', 
                  'end_date': 'object',
                  'assmt_taxes': 'float64',
                  'pilot_contract_amt': 'float64',
       'pilot_debt_service_amt': 'float64', 
                  'difference': 'float64'}
df = cast_data_types(df, names_to_types)
df.dtypes

owner                       string[python]
location                    string[python]
pilot_parcel_no             string[python]
underlying_parcel_number    string[python]
begin_date                  datetime64[ns]
end_date                            object
assmt_taxes                        float64
pilot_contract_amt                 float64
pilot_debt_service_amt             float64
difference                         float64
dtype: object

In [24]:
# Save the now-clean csv and convert it to utf-8 format
outfile_path = "../data/processed/s12_2021_idb_germantown.csv"
df.to_csv(outfile_path)
convert_utf8(original_file_path=outfile_path, new_file_path=outfile_path)